In [ ]:
# Gerekli paketleri yükleme
from google.colab import drive
drive.mount('/content/drive')

zip_adres = "/content/drive/My Drive/archive.zip"
!cp "{zip_adres}" .

!unzip -q archive.zip
#!rm archive.zip

!pip install timm
!pip install torch torchvision
!pip install numpy pandas matplotlib seaborn scikit-learn

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, random_split
from timm import create_model
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, matthews_corrcoef
from PIL import Image

# Veri seti yolları
data_dir = '/content/drive/MyDrive/MAKİNE ÖĞRENMESİ VERİLER/train'
categories = ['Çamurlu Zemin_TRAIN', 'KARSTİK ZEMİN_TRAIN', 'Kumlu Zemin_TRAIN', 'Sulak Zemin_TRAIN']

# Özel veri seti sınıfı
class CustomDataset(Dataset):
    def __init__(self, filepaths, labels, transform=None):
        self.filepaths = filepaths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.filepaths)

    def __getitem__(self, idx):
        img_path = self.filepaths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

# Transform işlemleri
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Dosya yollarını ve etiketleri elde etme
all_filepaths = []
all_labels = []
for i, category in enumerate(categories):
    category_dir = os.path.join(data_dir, category)
    for filename in os.listdir(category_dir):
        filepath = os.path.join(category_dir, filename)
        all_filepaths.append(filepath)
        all_labels.append(i)  # Integer formatında kategoriler

# K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
all_metrics = []

# PyTorch model sınıfı
class LeViTModel(torch.nn.Module):
    def __init__(self, num_classes):
        super(LeViTModel, self).__init__()
        self.backbone = create_model('levit_384', pretrained=True, num_classes=num_classes)

    def forward(self, x):
        return self.backbone(x)

# Eğitim ve doğrulama fonksiyonları
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(dataloader.dataset)
    return epoch_loss

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    epoch_loss = running_loss / len(dataloader.dataset)
    accuracy = accuracy_score(all_labels, all_preds)
    return epoch_loss, accuracy, all_preds, all_labels

# Eğitim süreci
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(categories)

for fold, (train_index, val_index) in enumerate(kf.split(all_filepaths)):
    print(f"Fold {fold + 1}")

    train_filepaths = np.array(all_filepaths)[train_index]
    train_labels = np.array(all_labels)[train_index]
    val_filepaths = np.array(all_filepaths)[val_index]
    val_labels = np.array(all_labels)[val_index]

    train_dataset = CustomDataset(train_filepaths, train_labels, transform=transform)
    val_dataset = CustomDataset(val_filepaths, val_labels, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    model = LeViTModel(num_classes=num_classes).to(device)
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

    num_epochs = 3
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, optimizer, device)
        val_loss, val_accuracy, val_preds, val_labels = validate(model, val_loader, criterion, device)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")


Mounted at /content/drive
cp: cannot stat '/content/drive/My Drive/archive.zip': No such file or directory
unzip:  cannot find or open archive.zip, archive.zip.zip or archive.zip.ZIP.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 11.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

Epoch 1/3, Train Loss: 1.0107, Val Loss: 1.9434, Val Accuracy: 0.7867
Epoch 2/3, Train Loss: 0.5249, Val Loss: 1.5394, Val Accuracy: 0.8706
Epoch 3/3, Train Loss: 0.3237, Val Loss: 0.2748, Val Accuracy: 0.9079
Fold 2
Epoch 1/3, Train Loss: 1.0157, Val Loss: 1.3889, Val Accuracy: 0.7712
Epoch 2/3, Train Loss: 0.5157, Val Loss: 0.3887, Val Accuracy: 0.8768
Epoch 3/3, Train Loss: 0.2875, Val Loss: 0.2901, Val Accuracy: 0.9068
Fold 3
Epoch 1/3, Train Loss: 1.0115, Val Loss: 0.6597, Val Accuracy: 0.7865
Epoch 2/3, Train Loss: 0.5315, Val Loss: 0.3823, Val Accuracy: 0.8881
Epoch 3/3, Train Loss: 0.2959, Val Loss: 1.8426, Val Accuracy: 0.9181
Fold 4
Epoch 1/3, Train Loss: 1.0169, Val Loss: 0.7364, Val Accuracy: 0.7130
Epoch 2/3, Train Loss: 0.5656, Val Loss: 1.0361, Val Accuracy: 0.8964
Epoch 3/3, Train Loss: 0.3415, Val Loss: 0.4065, Val Accuracy: 0.9026
Fold 5
Epoch 1/3, Train Loss: 0.9998, Val Loss: 0.6771, Val Accuracy: 0.7803
Epoch 2/3, Train Loss: 0.5060, Val Loss: 0.6561, Val Accuracy:

In [ ]:
# Epoch sayısına göre loss ve accuracy değerleri
epochs = range(1, num_epochs + 1)

# Loss grafiği
plt.figure(figsize=(12, 6))
plt.plot(epochs, train_losses, label='Training Loss')
plt.plot(epochs, val_losses, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss over Epochs')
plt.legend()
plt.show()
